The functions below implement the main theorem in "Characters of the nullcone" \(Math. Ann. volume 252 no. 3 \(1980\), pages 179\-182\) by Wim Hesselink.  For a semisimple algebraic group $G$ defined over $\mathbb{C}$ with Lie algebra $\mathfrak{g}$, let $N$ be the variety of nilpotent elements in $\mathfrak{g}$ \(the _nullcone_ of $\mathfrak{g}$\).  Let $A(N)$ be the coordinate ring of $N$ and $A_n(N)$ its homogeneous component of degree $n$.  If $E_\lambda$ is an irreducible $G$\-module of highest weight $\lambda$, then let $d_n(\lambda)$ be the multiplicity of $E_\lambda$ in $A_n(N)$.  Hesselink's formula computes the values of $d_n(\lambda)$ in terms of the Weyl group of $G$.


In [1]:
"""
The function `replacementrules` accepts a root system type as input and returns a list which indicates how
each non-simple positive root is written as a linear combination of the simple roots. This function is used
by the function `condensed` below.
"""

def replacementrules(Type):

    R= RootSystem(Type)
    L= R.ambient_space()
    posrts= L.positive_roots()
    simprts= L.simple_roots()
    simprtslist= list(simprts)
    nonsimp= list(set(posrts)-set(simprtslist))
    for i in range (0, len(simprtslist)):
        simprtslist[i]= list(vector(simprtslist[i]))
    M= column_matrix(simprtslist)
    solutions=[]
    for i in range (0, len(nonsimp)):
        y= vector(nonsimp[i])
        solutions= solutions + [M.solve_right(y)]
    rules=[]
    for i in range (0, len(nonsimp)):
        rules= rules+[[len(simprtslist) + i + 1, list(solutions[i])]]
    return rules


In [2]:
"""
The function `partition` is used to partition any natural number n into p parts. In our case,
we input an n which represents a homogeneous polynomial degree and a p which represents the number
of positive roots in any specified root system. This gives us the list L of all the partitions of n 
into p parts as output. This function is called by `condensed`.
"""

def partition(n,p):
    L= []
    if p==1:
        return[[n]]
    else:
        for i in range (0, n+1):
            M= partition(n-i, p-1)
            for j in range (0, len(M)):
                M[j]= [i]+ M[j]
            L= L+M
        return (L)

In [3]:
"""
The function `condensed` uses the function `replacementrules` to condense the list of partitions generated by 
`partition` by replacing the positive roots with linear combinations of simple roots and collecting like terms. 
It is called by the function `Kostant` below.
"""

def condensed (n,type):
    R= RootSystem(type)
    A= R.ambient_space()
    posrts= A.positive_roots()
    p= len(posrts)
    L= partition (n,p)
    CL= len(L)*[[]]
    i=0
    for q in L:
        rules= replacementrules(type)
        simprts= A.simple_roots()
        simprtslist= list(simprts)
        m=len(simprtslist)
        p1= vector(q[0:m])
        for j in range(0,len(rules)):
            p1 = p1 + q[rules[j][0]-1]*vector(rules[j][1])
        CL[i] = list(p1)
        i = i + 1
    return(CL)

In [4]:
"""
The function `frequencies` takes the output given by `condensed` and groups together repeated elements.
This returns a list of the condensed list with duplicated elements removed, along with the number of
times each repeated element occurs. This function is called upon by `Kostant` below.
"""

def frequencies(L):
    res=[]
    for k in L:
        if k not in res:
            res.append(k)
    sums=[]
    for i in range (0, len(res)):
        sum=0
        for j in range (0, len(L)):
            if L[j]==res[i]:
                sum=sum+1
        sums.append(sum)
    return [res,sums]

In [5]:
"""
The function `rho` calculates the half sum of the positive roots.  It is called by the `Waction` function.
"""

def rho(Type):
    R=RootSystem(Type)
    L=R.ambient_space()
    posrts=L.positive_roots()
    S= 1/2*sum(posrts)
    return S

In [6]:
"""
The function `Converttostandard` converts a weight written as a list of the coefficients of a linear combination
of the simple roots (which is convenient notation for our calculations) to Cartesian coordinates in a
Euclidean vector space (which is how SAGE expects weights to be denoted). This function is called by the
function `Waction`.
"""

def Converttostandard (Type, weight):
    R= RootSystem(Type)
    L=R.ambient_space()
    simprts= L.simple_roots()
    S=sum(c*v for c,v in zip(weight, simprts))
    return S

In [7]:
"""
`Converttocombination` reverses the conversion accomplished by `Converttostandard`.  It is also called upon by `Waction`.
"""

def Converttocombination (Type, weight):
    R= RootSystem(Type)
    L=R.ambient_space()
    simprts= L.simple_roots()
    simprtslist= list(simprts)
    for i in range(0, len(simprtslist)):
        simprtslist[i] = list(vector(simprtslist[i]))
    M=column_matrix(simprtslist)
    weight= vector(weight)
    Y=M.solve_right(weight)
    return list(Y)

In [9]:
"""
The function `Waction` computes the dot action of the Weyl group on the set of weights according to the
formula w(lambda + rho) - rho, where w is a Weyl group element, lambda is the weight, and rho is the previously calculated
half-sum of positive roots. This function is called upon by the `d_n` function.
"""

def Waction(w,weight,Type):
    R=rho(Type)
    C= Converttostandard(Type, weight)
    W=w.action(C+R)-R
    S= Converttocombination(Type,W)
    return S

In [10]:
"""
Given a root system type, any specific weight i, and an n-value which is a homogeneous polynomial degree, the `Kostant`
function tells us how many times a given weight shows up in the condensed list. It is called by the `d_n`
function.  (This is known in the literature as the "Kostant partition function".)
"""

def Kostant (type, i, n):
    B= condensed(n, type)
    F = frequencies(B)
    if i in F[0]:
        return F[1][F[0].index(i)]
    else:
        return 0

In [11]:
"""
The function `is_dom` returns a value of 'True' if the input weight is in the dominant chamber relative to the
input root system type. It is called by the function `check_all`.
"""

def is_dom(y, type):
    M = CartanMatrix(type)
    y = vector(y)
    s = sorted(list(M*y))
    if s[0] < 0:
        return False
    else:
        return True

In [12]:
"""
Given a root system type, a weight, and an n which is the degree of a homogeneous polynomial, the 
function `d_n` computes the number of times an irreducible highest weight representation
shows up in the decomposition of A_n[N].  This function is only defined for weights in the dominant chamber 
of any given root system. This function is called upon by the `check_all` function, and it is how we finally 
implement Hesselink’s multiplicity formula.
"""

def d_n(Type,weight,n):
    W= WeylGroup(Type)
    S= sum(((-1)^W[i].length())*(Kostant(Type, Waction(W[i], weight, Type), n)) for i in range(0, W.order()))
    return S

In [13]:
"""
Given a root system type and a degree n, the function `check_all` returns a list of all of the weights in the
dominant chamber with nonzero d_n values paired with their d_n values. It is called by the `decompose`
function.
"""

def check_all(type, n):
    W = WeylGroup(type)
    nonzeropn = condensed(n, type)
    A = []
    for p in nonzeropn:
        for w in list(W):
            v = Waction(w, p, type)
            if is_dom(v, type) and not(v in A):
                A = A + [v]
    T = []
    for j in A:
        T = T + [[d_n(type, j, n),j]]
    return T

In [14]:
"""
The function `decompose` returns the weights which determine the highest weight irreducible components
of the representation A_n[N], given a root system type and a homogeneous degree n. This function accomplishes
the decomposition of the representation into irreducible components, which is the goal of this
project.
"""

def decompose(type, n):
    components = []
    L = check_all(type, n)
    for S in L:
        for j in range(0,S[0]):
            components.append(S[1])
    return components

In [15]:
"""
We can now compute the structure of A_n[N] as a representation of a given group and given n-value.  The command below executes 
Hesselink's formula for the group SL_2(C) when n=2.  The output [[2]] indicates that there is only one irreducible component 
of the representation A_2[N], and it's the irreducible SL_2(C)-module of highest weight 2a, where a is the simple root in Type 
A1.  This means the representation is irreducible.
"""

decompose("A1",2)

[[2]]

In [17]:
"""
The following also shows that A_n[N] is irreducible for n=3 and n=4.  In fact, it's easy to prove, using Hesselink's formula, that 
A_n[N] is an irreducible representation of SL_2(C) for all n.
"""

decompose("A1",3)

[[3]]

In [18]:

decompose("A1",4)

[[4]]

In [19]:
"""
We can determine the structure of A_n[N] for other types and n-values as well.  The following shows that A_3[N] decomposes into four 
irreducible components as an Sp_4(C)-module.  It's components are the irreducible highest weight modules of weights 4a+3b, 6a+3b, 3a+2b, 
and 2a+b, where a is the short simple root in C_2 and b is the long simple root.
"""

decompose("C2",3)

[[4, 3], [6, 3], [3, 2], [2, 1]]